In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data
data = pd.read_csv('data_clean.csv')

# Let's assume you want an 80-20 train-test split
train_data, test_data = train_test_split(data, test_size=0.01, shuffle=False)

# Save them if you want
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (928988, 7)
Test shape: (9384, 7)


In [30]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from tqdm import tqdm


def train_model(
    train_data: pd.DataFrame,
    lags: int
) -> ARIMA:
    model = ARIMA(
        train_data,
        order=(lags, 1, 2),                # <-- d=1 here
        enforce_stationarity=False,        # relax stationarity check
        enforce_invertibility=False        # relax invertibility check
    )
    return model.fit()


def evaluate_model(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    minutes: int = 4 * 60,
    lags: int = 2
) -> None:
    mses = []

    num_batch = len(test_data) // minutes - 1

    for batch in tqdm(range(num_batch), total=num_batch):
        
        model_fit = train_model(pd.concat([train_data, test_data[ : batch * minutes]]), lags)
        
        forecast = model_fit.forecast(steps=minutes)

        # Evaluate the forecast
        mse = mean_squared_error(test_data.iloc[batch * minutes : (batch + 1) * minutes], forecast)
        
        print(f'Batch ({batch}): Mean Squared Error on test data: {mse:.4f}')
        
        mses.append(mse)
        
    print(f"Average mean squared error on test data: {np.mean(mses)}")


In [31]:
train_data = pd.read_csv('train.csv', parse_dates=["Date"], index_col=0)["2013-01-01 00:00:00":]
train_data = train_data.asfreq(pd.infer_freq(train_data.index))

test_data = pd.read_csv('test.csv', parse_dates=["Date"], index_col=0)
test_data = test_data.asfreq(pd.infer_freq(test_data.index))

evaluate_model(
    train_data=train_data.iloc[:, 4],
    test_data=test_data.iloc[:, 4]
)

  3%|▎         | 1/38 [00:39<24:25, 39.61s/it]

Batch (0): Mean Squared Error on test data: 0.8315


  5%|▌         | 2/38 [01:05<18:57, 31.60s/it]

Batch (1): Mean Squared Error on test data: 0.1392


  8%|▊         | 3/38 [01:40<19:20, 33.17s/it]

...

Batch (35): Mean Squared Error on test data: 2.0982


 97%|█████████▋| 37/38 [28:20<00:49, 49.92s/it]

Batch (36): Mean Squared Error on test data: 0.6055


100%|██████████| 38/38 [28:44<00:00, 45.38s/it]

Batch (37): Mean Squared Error on test data: 3.7299
Average mean squared error on test data: 1.0056627875776178
